<a href="https://colab.research.google.com/github/ankitdv98/Fake-News-Classifier/blob/main/Fake_News_Classifier_with_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!ls

 appdata10.csv		       ipl_data			   test.csv
 car_brand_model_resnet50.h5   kaggle.json		   test.csv.zip
 creditcardfraud.zip	       lasso_regressor.pickle	   top_screens.csv
 final_submit.csv	      'Movie Sentiment Analysis'   train.csv
 flight_rf.pkl		       submit.csv		   train.csv.zip


In [29]:
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
df= pd.read_csv('train.csv')

In [32]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
df.shape

(20800, 5)

In [34]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [35]:
df.drop('author', axis= 1, inplace= True)

In [36]:
df.dropna(inplace= True)
df.reset_index(inplace= True)

In [37]:
df.shape

(20203, 5)

In [38]:
import nltk
nltk.download('popular')
import re


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [39]:
y= df['label']

In [40]:
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps= PorterStemmer()
corpus= []
for i in range(len(df)):
  text= re.sub('[^a-zA-Z]', ' ', df.loc[i, 'title'])
  text= text.lower()
  text= text.split()
  words= [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
  sent= " ".join(words)
  corpus.append(sent)

In [41]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [42]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [44]:
voc_size= 20000
onehot_repr= [one_hot(words, voc_size) for words in corpus]
onehot_repr[0:5]

[[8244, 7679, 10280, 11355, 18256, 1233, 14344, 17692, 9476, 4267],
 [2106, 14844, 13146, 16196, 1664, 17469, 14412],
 [264, 9409, 13690, 1527],
 [7695, 8448, 4326, 2913, 16103, 1485],
 [18594, 1664, 16696, 8502, 10911, 3362, 1664, 694, 19691, 11151]]

#Embedding Representation

In [47]:
sent_length= 20
embed_doc= pad_sequences(onehot_repr, maxlen= sent_length, padding= 'pre')
embed_doc[0:5]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,  8244,  7679, 10280, 11355, 18256,  1233, 14344, 17692,
         9476,  4267],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,  2106, 14844, 13146, 16196,  1664,
        17469, 14412],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,   264,  9409,
        13690,  1527],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,  7695,  8448,  4326,  2913,
        16103,  1485],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 18594,  1664, 16696,  8502, 10911,  3362,  1664,   694,
        19691, 11151]], dtype=int32)

In [48]:
embed_doc[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,  8244,  7679, 10280, 11355, 18256,  1233, 14344, 17692,
        9476,  4267], dtype=int32)

In [50]:
import numpy as np
X=np.array(embed_doc)
y=np.array(y)

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 1)

In [52]:
X_train.shape, y_train.shape

((16162, 20), (16162,))

# Bidirectional LSTM Model

In [81]:
feature_dim= 100
model= Sequential()
model.add(Embedding(voc_size, feature_dim, input_length= sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation= 'sigmoid'))
model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 100)           2000000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 2,161,001
Trainable params: 2,161,001
Non-trainable params: 0
_________________________________________________________________


In [82]:
model.fit(X_train, y_train, validation_data= (X_test, y_test), epochs= 10, batch_size= 64)

Epoch 1/10
253/253 [==============================] - 9s 26ms/step - loss: 0.3827 - accuracy: 0.8044 - val_loss: 0.1858 - val_accuracy: 0.9312
Epoch 2/10
253/253 [==============================] - 6s 23ms/step - loss: 0.1094 - accuracy: 0.9590 - val_loss: 0.1566 - val_accuracy: 0.9394
Epoch 3/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0561 - accuracy: 0.9797 - val_loss: 0.1765 - val_accuracy: 0.9379
Epoch 4/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0360 - accuracy: 0.9889 - val_loss: 0.2526 - val_accuracy: 0.9379
Epoch 5/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.2845 - val_accuracy: 0.9287
Epoch 6/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0154 - accuracy: 0.9952 - val_loss: 0.3142 - val_accuracy: 0.9312
Epoch 7/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0083 - accuracy: 0.9981 - val_loss: 0.3317 - val_accuracy: 0.9292

# Performance Metrics

In [83]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [84]:
pred= (model.predict(X_test) > 0.5).astype("int32")
score= accuracy_score(y_test, pred)
print("accuracy score: ", score)
cm= confusion_matrix(y_test, pred)
print(cm)

accuracy score:  0.9284830487503093
[[1934  154]
 [ 135 1818]]


In [85]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93      2088
           1       0.92      0.93      0.93      1953

    accuracy                           0.93      4041
   macro avg       0.93      0.93      0.93      4041
weighted avg       0.93      0.93      0.93      4041



# Improving Model
Adding more dense layer

In [86]:
feature_dim= 100
model= Sequential()
model.add(Embedding(voc_size, feature_dim, input_length= sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(100, activation= 'relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation= 'sigmoid'))
model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 100)           2000000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 2,181,001
Trainable params: 2,181,001
Non-trainable params: 0
____________________________________________

In [87]:
model.fit(X_train, y_train, validation_data= (X_test, y_test), epochs= 10, batch_size= 64)

Epoch 1/10
253/253 [==============================] - 9s 26ms/step - loss: 0.3798 - accuracy: 0.8166 - val_loss: 0.1582 - val_accuracy: 0.9339
Epoch 2/10
253/253 [==============================] - 6s 23ms/step - loss: 0.1046 - accuracy: 0.9615 - val_loss: 0.1603 - val_accuracy: 0.9342
Epoch 3/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0643 - accuracy: 0.9773 - val_loss: 0.1792 - val_accuracy: 0.9344
Epoch 4/10
253/253 [==============================] - 6s 24ms/step - loss: 0.0342 - accuracy: 0.9892 - val_loss: 0.2304 - val_accuracy: 0.9334
Epoch 5/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0196 - accuracy: 0.9943 - val_loss: 0.3077 - val_accuracy: 0.9324
Epoch 6/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0135 - accuracy: 0.9961 - val_loss: 0.3831 - val_accuracy: 0.9290
Epoch 7/10
253/253 [==============================] - 6s 23ms/step - loss: 0.0071 - accuracy: 0.9977 - val_loss: 0.3394 - val_accuracy: 0.9282

# Performance Metrics

In [88]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [89]:
pred= (model.predict(X_test) > 0.5).astype("int32")
score= accuracy_score(y_test, pred)
print("accuracy score: ", score)
cm= confusion_matrix(y_test, pred)
print(cm)

accuracy score:  0.9289779757485771
[[1935  153]
 [ 134 1819]]


In [90]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      2088
           1       0.92      0.93      0.93      1953

    accuracy                           0.93      4041
   macro avg       0.93      0.93      0.93      4041
weighted avg       0.93      0.93      0.93      4041

